# 📘 Meu portfólio de dados ambientais
## Como os estados brasileiros emitem gases de efeito estufa

Sou David e construí este notebook para apresentar, de maneira pessoal, como analiso grandes bases públicas. Quero mostrar minha capacidade de:

1. Organizar um fluxo analítico claro.
2. Limpar e preparar dados reais, tratando nulos e outliers.
3. Contar uma boa história a partir dos números.

A base utilizada vem do SEEG (Sistema de Estimativa de Emissões e Remoções de Gases de Efeito Estufa) e cobre emissões por estado brasileiro de 1970 a 2021.

### Roteiro do estudo
1. Importar bibliotecas e configurar o ambiente.
2. Carregar os dados originais e entender sua estrutura.
3. Limpar a base (tratamento de nulos) e remodelar para o formato longo.
4. Suavizar outliers de emissões anuais por estado usando IQR.
5. Construir visualizações que contem a evolução nacional, os estados líderes e os setores determinantes.
6. Encerrar com lições e próximos passos.

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

sns.set_theme(style="whitegrid", palette="crest")
plt.rcParams["figure.figsize"] = (12, 6)
pd.set_option("display.max_columns", 30)
pd.options.display.float_format = "{:.2f}".format

### 1. Carregamento dos dados
A planilha `dados_gases.xlsx` está no diretório do projeto e contém a aba **GEE Estados**, com o inventário anual de emissões por estado e por tipo de atividade econômica.

In [ ]:
DATA_PATH = Path("dados_gases.xlsx")
if not DATA_PATH.exists():
    raise FileNotFoundError(
        "Arquivo 'dados_gases.xlsx' não encontrado. Certifique-se de mantê-lo na raiz do projeto."
    )

emissoes_raw = pd.read_excel(DATA_PATH, sheet_name="GEE Estados")
print(f"A base original possui {emissoes_raw.shape[0]:,} linhas e {emissoes_raw.shape[1]} colunas.")
emissoes_raw.sample(5, random_state=42)

### 2. Diagnóstico rápido de valores nulos
Quais colunas mais sofrem com ausência de informação?

In [ ]:
null_report = (
    emissoes_raw.isna()
    .mean()
    .sort_values(ascending=False)
    .mul(100)
    .round(1)
    .to_frame("Percentual de nulos (%)")
)
null_report.head(12)

### Decisões de limpeza
- **Produto**: mais de 60% de nulos e pouco valor para a narrativa → coluna descartada.
- **Níveis 3 a 6**: completados com o rótulo *"Não informado"* para preservar a hierarquia mesmo sem detalhe.
- **Estado**: entradas agregadas (NaN) são rotuladas como *"BRASIL"* para diferenciar dos estados.
- **Atividade Econômica**: completada com *"Não informado"* quando vazia.
- **Filtro**: mantemos somente linhas com `Emissão / Remoção / Bunker == 'Emissão'`, removendo remoções, bunkers e emissões fora do inventário nacional.

In [ ]:
cols_to_drop = ["Produto"]
fill_defaults = {
    "Estado": "BRASIL",
    "Atividade Econômica": "Não informado",
    "Nível 3": "Não informado",
    "Nível 4": "Não informado",
    "Nível 5": "Não informado",
    "Nível 6": "Não informado",
}

emissoes = (
    emissoes_raw.loc[emissoes_raw["Emissão / Remoção / Bunker"] == "Emissão"]
    .drop(columns=cols_to_drop)
    .copy()
)

demographic_cols = list(fill_defaults.keys())
emissoes[demographic_cols] = emissoes[demographic_cols].fillna(fill_defaults)

emissoes = emissoes.drop(columns=["Emissão / Remoção / Bunker"])
emissoes = emissoes.rename(columns={"Gás": "Gás (referência)"})

print(
    f"Base após limpeza: {emissoes.shape[0]:,} linhas e {emissoes.shape[1]} colunas."
)

(
    emissoes.isna()
    .mean()
    .sort_values(ascending=False)
    .head(5)
)

### 3. Remodelando para o formato longo
Para somar emissões por período, precisamos condensar as colunas de 1970–2021 em uma única coluna `Ano`.

In [ ]:
info_cols = [col for col in emissoes.columns if not isinstance(col, int)]
year_cols = sorted([col for col in emissoes.columns if isinstance(col, int)])

emissoes_long = emissoes.melt(
    id_vars=info_cols,
    value_vars=year_cols,
    var_name="Ano",
    value_name="Emissao"
)

emissoes_long["Ano"] = emissoes_long["Ano"].astype(int)
emissoes_long.head()

### 4. Tratamento de outliers com IQR
Emissões estaduais podem ter saltos bruscos (queimadas específicas, por exemplo). Para manter a tendência geral sem perder sinal, apliquei um *winsorize* baseado no intervalo interquartílico (IQR) por ano.

In [ ]:
def winsorize_iqr(series: pd.Series) -> pd.Series:
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    if iqr == 0:
        return series
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return series.clip(lower, upper)

emissoes_estado = (
    emissoes_long.groupby(["Estado", "Ano"], as_index=False)["Emissao"].sum()
    .query("Estado != 'BRASIL'")
)

emissoes_estado["Emissao_sem_outlier"] = (
    emissoes_estado.groupby("Ano")["Emissao"].transform(winsorize_iqr)
)

impacto_outlier = pd.DataFrame({
    "Total sem tratamento": [emissoes_estado["Emissao"].sum()],
    "Total suavizado": [emissoes_estado["Emissao_sem_outlier"].sum()]
})
impacto_outlier.assign(
    Redução_percentual=lambda df: (1 - (df["Total suavizado"] / df["Total sem tratamento"])) * 100
)

### 5. Storytelling visual
#### 5.1 Evolução das emissões nacionais

In [ ]:
totais_ano = (
    emissoes_estado.groupby("Ano", as_index=False)["Emissao_sem_outlier"].sum()
)

fig, ax = plt.subplots()
sns.lineplot(data=totais_ano, x="Ano", y="Emissao_sem_outlier", ax=ax, linewidth=2)
ax.set_title("Emissões totais suavizadas por ano (Estados brasileiros)")
ax.set_ylabel("Emissões (toneladas equivalentes de CO₂)")
ax.yaxis.set_major_formatter(
    FuncFormatter(lambda value, pos: f"{value/1e6:.1f} mi t")
)
ax.set_xlabel("Ano")
plt.tight_layout()
plt.show()

> **Insight:** mesmo após suavizar outliers, a curva nacional mostra queda no início da década de 2000 e nova escalada recente, refletindo pressões sobre mudança de uso da terra.

#### 5.2 Estados que mais emitiram em 2021

In [ ]:
top_2021 = (
    emissoes_estado.loc[emissoes_estado["Ano"] == 2021]
    .nlargest(10, "Emissao_sem_outlier")
    .sort_values("Emissao_sem_outlier", ascending=True)
)

fig, ax = plt.subplots()
sns.barplot(
    data=top_2021,
    x="Emissao_sem_outlier",
    y="Estado",
    palette="viridis",
    ax=ax
)
ax.set_title("Top 10 estados emissores em 2021")
ax.set_xlabel("Emissões suavizadas (t de CO₂e)")
ax.set_ylabel("")
ax.xaxis.set_major_formatter(
    FuncFormatter(lambda value, pos: f"{value/1e6:.1f} mi")
)
plt.tight_layout()
plt.show()

> **Insight:** Mato Grosso, Pará e Amazonas lideram o ranking recente — evidência clara do peso do desmatamento amazônico nas emissões nacionais.

#### 5.3 Quais setores puxam as emissões?

In [ ]:
emissoes_setor = (
    emissoes_long.groupby(["Nível 1 - Setor", "Ano"], as_index=False)["Emissao"].sum()
)

top_setores = (
    emissoes_setor.groupby("Nível 1 - Setor")["Emissao"].sum().nlargest(4).index
)

emissoes_setor_top = emissoes_setor[emissoes_setor["Nível 1 - Setor"].isin(top_setores)]

fig, ax = plt.subplots()
sns.lineplot(
    data=emissoes_setor_top,
    x="Ano",
    y="Emissao",
    hue="Nível 1 - Setor",
    ax=ax,
    linewidth=2
)
ax.set_title("Setores que mais contribuem para as emissões")
ax.set_ylabel("Emissões (t de CO₂e)")
ax.yaxis.set_major_formatter(
    FuncFormatter(lambda value, pos: f"{value/1e6:.1f} mi")
)
ax.legend(title="Setor", bbox_to_anchor=(1.02, 1), loc="upper left")
plt.tight_layout()
plt.show()

> **Insight:** Mudança de uso da terra domina o panorama, seguida de Agropecuária e Energia. Os saltos recentes em "Mudança de Uso da Terra" acontecem ao mesmo tempo em que os estados amazônicos aparecem no topo, reforçando a narrativa.

### 6. Conclusões e próximos passos
- **Narrativa construída:** tratamos nulos, suavizamos outliers e mostramos como emissões nacionais ainda dependem da agenda de desmatamento.
- **Estados críticos:** MT e PA concentram emissões recentes; merecem planos locais específicos.
- **Setores críticos:** Mudança de uso da terra segue como principal vetor, mas agropecuária mantém patamar elevado.

**Próximos passos que pretendo atacar:**
1. Enriquecer a análise com dados econômicos (PIB/produção agropecuária) para medir intensidade de emissões.
2. Criar um dashboard interativo (Streamlit ou Power BI) reaproveitando os datasets já limpos.
3. Explorar emissões per capita integrando dados populacionais por estado.

> Notebook produzido para fins de portfólio pessoal. Fonte dos dados: [SEEG](http://seeg.eco.br/download).

### 7. Exportando a base para o dashboard
Para facilitar a construção de painéis interativos, exporto abaixo a base já filtrada e suavizada (`emissoes_estado`). Ela contém as emissões anuais por estado, prontas para conexão em soluções como Power BI ou Streamlit.

In [ ]:
OUTPUT_CSV = Path("emissoes_estado_filtrado.csv")
emissoes_estado.to_csv(OUTPUT_CSV, index=False)
print(f"Arquivo para dashboard salvo em: {OUTPUT_CSV.resolve()}")